In [1]:
import geemap
import ee

In [2]:
# ee.Authenticate()

ee.Initialize()

In [18]:
# Define the region of interest
roi = ee.Geometry.Rectangle(76.9844, 9.5178, 77.2894, 9.996260)

# Define the sentinal image collection and filter by date
dataset = ee.ImageCollection("COPERNICUS/S3/OLCI")\
     .select(['Oa08_radiance'])\
    .filterBounds(roi)\
    .filterDate('2022-01-01', '2022-01-31')


# function mask cloud
def mask_cloud(image):
    qa=image.select('QA60')
    cloudBitMask=1<<10
    cirrusBitMask=1<<11
    mask=qa.bitwiseAnd(cloudBitMask).eq(0).And(
        qa.bitwiseAnd(cirrusBitMask).eq(0)
    )
    return image.updateMask(mask)\
    .select('Oa08_radiance')\
    .copyProperties(image,['system:time_start'])


   
filterd=dataset.map(mask_cloud)
# Convert to radiance units
conversion_factors = ee.Image([0.00876539])
radiance = filterd.first().multiply(conversion_factors)    



In [19]:
image_ids=filterd.aggregate_array('system:index').getInfo()
len(image_ids)

69

### Download images

In [ ]:
for i , image_id in enumerate(image_ids):
    image=